## **Deep Learning - Ex-1**

Tomer Zamir  
ID: 316123181

In [ ]:
%load_ext tensorboard
import datetime, os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras

### Load a dataset


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist_corrupted',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True, 
)

the_shape = (28, 28, 1)

### Build a training pipeline


In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)
ds_train

### Build an evaluation pipeline


In [ ]:
ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

Initialization with he_normal

Applying 7 layers each with 512 neurons.






In [ ]:
layers = [
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dense(10,  kernel_initializer='he_normal')
]

Default with initialization and dropout 0.4

In [ ]:
layers = [
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(10, kernel_initializer='he_normal')
]

initialization with he_normal and BN


In [ ]:
layers = [
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(784, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(512, kernel_initializer='he_normal', activation='relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(10, kernel_initializer='he_normal')
]


Init randomly with std=0.01

In [ ]:
layers = [
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(784, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.Dense(10, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01))
  ]

Dropout + STD 0.01

In [ ]:
layers = [
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(784, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Dense(10, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01))
  ]

**Chosen Model Layers:**<br>
Batch Normalization and STD 0.01

In [ ]:
layers = [
  tf.keras.layers.Flatten(input_shape=(28,28)),
  tf.keras.layers.Dense(784, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Dense(10, kernel_initializer=tf.keras.initializers.RandomNormal(mean=0.0, stddev=0.01))
  ]

Compile the model

In [ ]:

model = tf.keras.models.Sequential(layers)


model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

model.summary()

Create Tensorboard 

In [ ]:
%tensorboard --logdir logs
from tensorboard import notebook
notebook.list() # View open TensorBoard instances


Train the model

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S")), histogram_freq=1) 
model.fit(ds_train, 
          epochs=10, 
          validation_data=ds_test, 
          callbacks=[tensorboard_callback],
          use_multiprocessing=True)
